In [ ]:
!git clone https://github.com/gbend22/Bioinformatics-Final-Assignment.git
import sys
sys.path.append('/content/Bioinformatics-Final-Assignment')

from google.colab import drive
drive.mount('/content/drive')

EXPR_PATH      = "/content/drive/MyDrive/tcga_RSEM_gene_fpkm.gz"
PHENOTYPE_PATH = "/content/drive/MyDrive/tcga_phenotype.tsv.gz"
PC_URL  = "https://raw.githubusercontent.com/CBIIT/TULIP/main/gene_lists/protein_coding_genes.txt"
PC_PATH = "/content/drive/MyDrive/protein_coding_genes.txt"

In [ ]:
from preprocessing import (
    load_protein_coding_genes,
    load_phenotype,
    load_expression_data,
    align_samples,
    filter_to_tulip_types,
    normalize_and_pad,
    encode_labels,
    split_data,
    create_dataloaders,
)
from models import LSTMClassifier

In [ ]:
pc_gene_ids = load_protein_coding_genes(PC_URL, PC_PATH)
labels_full, primary_tumor_idx = load_phenotype(PHENOTYPE_PATH)
expr_full = load_expression_data(EXPR_PATH, pc_gene_ids, primary_tumor_idx)
expr, labels = align_samples(expr_full, labels_full, primary_tumor_idx)
del expr_full

In [ ]:
expr, y_raw, labels = filter_to_tulip_types(expr, labels)
X = normalize_and_pad(expr)
del expr

y, le, NUM_CLASSES = encode_labels(y_raw)
NUM_GENES = X.shape[1]

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = split_data(X, y)
del X

train_loader, val_loader, test_loader = create_dataloaders(
    X_train, y_train, X_val, y_val, X_test, y_test
)

In [ ]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

SEQ_LEN    = 198
INPUT_SIZE = NUM_GENES // SEQ_LEN
PROJ_DIM   = SEQ_LEN * INPUT_SIZE

model = LSTMClassifier(
    num_genes   = NUM_GENES,
    seq_len     = SEQ_LEN,
    input_size  = INPUT_SIZE,
    hidden_size = 128,
    num_layers  = 2,
    num_classes = NUM_CLASSES,
    dropout     = 0.3,
).to(device)

In [ ]:
criterion  = nn.CrossEntropyLoss()
optimizer  = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-5)
scheduler  = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)

EPOCHS              = 60
EARLY_STOP_PATIENCE = 10

best_val_acc     = 0.0
patience_counter = 0
train_losses     = []
val_accs         = []

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out  = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)

    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds  = model(xb).argmax(dim=1)
            correct += (preds == yb).sum().item()
            total   += yb.size(0)
    val_acc = correct / total
    val_accs.append(val_acc)
    scheduler.step(1 - val_acc)

    print(f"Epoch {epoch:3d} | Loss: {avg_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_lstm.pt")
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= EARLY_STOP_PATIENCE:
            print(f"Early stopping at epoch {epoch}")
            break

print(f"Best validation accuracy: {best_val_acc:.4f}")

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

model.load_state_dict(torch.load("best_lstm.pt"))
model.eval()

all_preds, all_true = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        preds = model(xb).argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_true.extend(yb.numpy())

all_preds = np.array(all_preds)
all_true  = np.array(all_true)

test_acc = np.mean(all_preds == all_true)
print(f"Test Accuracy: {test_acc:.4f}")
print("Classification Report:")
print(classification_report(all_true, all_preds, target_names=le.classes_, zero_division=0))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(train_losses, color="steelblue", linewidth=2)
axes[0].set_xlabel("Epoch");  axes[0].set_ylabel("Loss")
axes[0].set_title("LSTM \u2014 Training Loss");  axes[0].grid(True)

axes[1].plot(val_accs, color="darkorange", linewidth=2)
axes[1].axhline(y=best_val_acc, color="red", linestyle="--",
                label=f"Best ({best_val_acc:.4f})")
axes[1].set_xlabel("Epoch");  axes[1].set_ylabel("Accuracy")
axes[1].set_title("LSTM \u2014 Validation Accuracy")
axes[1].legend();  axes[1].grid(True)

plt.tight_layout()
plt.savefig("lstm_training_curves.png", dpi=150)
plt.show()

cm = confusion_matrix(all_true, all_preds)

plt.figure(figsize=(20, 16))
sns.heatmap(cm, annot=True, fmt="d", cmap="YlOrRd",
            xticklabels=le.classes_, yticklabels=le.classes_,
            linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.xlabel("Predicted", fontsize=13)
plt.ylabel("Actual",    fontsize=13)
plt.title("LSTM \u2014 Confusion Matrix", fontsize=15)
plt.xticks(rotation=45, ha="right", fontsize=7.5)
plt.yticks(rotation=0,  fontsize=7.5)
plt.tight_layout()
plt.savefig("lstm_confusion_matrix.png", dpi=150)
plt.show()

precision = precision_score(all_true, all_preds, average=None, zero_division=0)
recall    = recall_score(   all_true, all_preds, average=None, zero_division=0)
f1        = f1_score(       all_true, all_preds, average=None, zero_division=0)

x_pos = np.arange(len(le.classes_))
width = 0.27

fig, ax = plt.subplots(figsize=(20, 7))
ax.bar(x_pos - width, precision, width, label="Precision", color="steelblue")
ax.bar(x_pos,         recall,   width, label="Recall",    color="darkorange")
ax.bar(x_pos + width, f1,       width, label="F1 Score",  color="seagreen")

ax.set_xticks(x_pos)
ax.set_xticklabels(le.classes_, rotation=45, ha="right", fontsize=7.5)
ax.set_ylabel("Score", fontsize=12)
ax.set_title("LSTM \u2014 Per-Class Precision / Recall / F1", fontsize=14)
ax.axhline(y=test_acc, color="red", linestyle="--", linewidth=1.2,
           label=f"Overall Acc ({test_acc:.4f})")
ax.legend(fontsize=11);  ax.set_ylim(0, 1.1);  ax.grid(axis="y")
plt.tight_layout()
plt.savefig("lstm_per_class_metrics.png", dpi=150)
plt.show()

fig, ax = plt.subplots(figsize=(20, 3))
sns.heatmap(f1.reshape(1, -1), annot=True, fmt=".2f", cmap="RdYlGn",
            vmin=0, vmax=1, xticklabels=le.classes_, ax=ax,
            cbar_kws={"shrink": 0.6})
ax.set_yticklabels(["LSTM"], fontsize=11)
ax.set_xticklabels(le.classes_, rotation=45, ha="right", fontsize=7.5)
ax.set_title("LSTM \u2014 Per-Class F1 Score", fontsize=14)
plt.tight_layout()
plt.savefig("lstm_f1_heatmap.png", dpi=150)
plt.show()

In [ ]:
import os

SAVE_DIR = "/content/drive/MyDrive/lstm_checkpoints"
os.makedirs(SAVE_DIR, exist_ok=True)

checkpoint = {
    "model_state_dict":     model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "best_val_acc":         best_val_acc,
    "test_acc":             float(test_acc),
    "train_losses":         train_losses,
    "val_accs":             val_accs,
    "hyperparams": {
        "NUM_GENES":    NUM_GENES,
        "PROJ_DIM":     PROJ_DIM,
        "SEQ_LEN":      SEQ_LEN,
        "INPUT_SIZE":   INPUT_SIZE,
        "hidden_size":  128,
        "num_layers":   2,
        "dropout":      0.3,
        "num_classes":  NUM_CLASSES,
    }
}
torch.save(checkpoint, os.path.join(SAVE_DIR, "best_lstm_checkpoint.pt"))

np.savez(
    os.path.join(SAVE_DIR, "shared_splits.npz"),
    X_train=X_train, X_val=X_val, X_test=X_test,
    y_train=y_train, y_val=y_val, y_test=y_test
)

np.save(os.path.join(SAVE_DIR, "label_classes.npy"), le.classes_)

lstm_results = {
    "test_acc":       float(test_acc),
    "best_val_acc":   float(best_val_acc),
    "train_losses":   np.array(train_losses),
    "val_accs":       np.array(val_accs),
    "all_preds":      all_preds,
    "all_true":       all_true,
    "precision":      precision,
    "recall":         recall,
    "f1":             f1,
    "classes":        le.classes_,
    "num_params":     sum(p.numel() for p in model.parameters()),
}
np.savez(os.path.join(SAVE_DIR, "lstm_results.npz"), **lstm_results)

print(f"LSTM Final Summary")
print(f"Best Val Accuracy :{best_val_acc:.4f}")
print(f"Test Accuracy:{test_acc:.4f}")
print(f"Macro F1:{f1.mean():.4f}")
print(f"Weighted F1:{np.average(f1, weights=[np.sum(all_true == i) for i in range(NUM_CLASSES)]):.4f}")
print(f"Parameters:{lstm_results['num_params']:,}")
print(f"Num Genes (input):{NUM_GENES}")
print(f"Num Classes:{NUM_CLASSES}")